In [1]:
import pandas as pd
import spacy
import stanza
import spacy
from stanza.utils.conll import CoNLL
import os
import re

In [2]:
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse')
nlp_spacy = spacy.load("es_core_news_md")

2025-03-05 17:35:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-05 17:35:14 INFO: Downloaded file to /home/ivan/stanza_resources/resources.json
2025-03-05 17:35:15 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

2025-03-05 17:35:15 INFO: Using device: cpu
2025-03-05 17:35:15 INFO: Loading: tokenize
2025-03-05 17:35:17 INFO: Loading: mwt
2025-03-05 17:35:17 INFO: Loading: pos
2025-03-05 17:35:19 INFO: Loading: lemma
2025-03-05 17:35:21 INFO: Loading: depparse
2025-03-05 17:35:21 INFO: Loading: ner
2025-03-05 17:35:24 INFO: Done loading processors!


In [3]:
baselineFilePath = "/home/ivan/Desktop/Uni/TFG/RepositorioTFG/Datasets/UD_Spanish-AnCora/es_ancora-ud-test.conllu"
baseline = CoNLL.conll2doc(baselineFilePath)
sentences = [i.text for i in baseline.sentences[:]]

# SPACY MODEL
spacy_output = nlp_spacy(" ".join(sentences))

# STANZA MODEL
stanza_output = nlp_stanza(" ".join(sentences))

In [4]:
# NOTE: Execute this only if you have installed Freeling!!
# FREELING MODEL
freeling_output_path = os.path.abspath("")+"/freelingOutput.conll" #Path for the output of freeling, change this if you want your output to be saved somewhere else
_tmp = open(os.path.abspath("")+"/tmp.txt", "w+").write(" ".join(sentences))
os.system(f"bash ./callFreeling.sh {os.path.abspath("")+"/tmp.txt"} {freeling_output_path}")
os.remove("tmp.txt")
freeling_parser = open(freeling_output_path, "r").read().split("\n\n")
freeling_parser = [i.split("\n") for i in freeling_parser]
columns = "ID FORM LEMMA TAG SHORT_TAG MSD NEC SENSE SYNTAX DEPHEAD DEPREL COREF SRL".split(" ")
freeling_parser = [[[{columns[i] : re.split(" +", word)[i]} for i in range(len(re.split(" +", word)))] for word in sent]for sent in freeling_parser]

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/freelingOutput.conll


In [5]:
# NOTE: Execute this only if you have installed Malt Parser!!
# NOTE: Also check that the Malt Parser folder is in the same directory as the callMaltParser.sh script
# MALT PARSER: chances are this will be removed, as I can't get it to work properly and it depends on freeling sentencizer and tagger
if baselineFilePath[len(baselineFilePath)-6:] != ".conll":
    os.system(f"python conllu-to-conll-standard.py {baselineFilePath} {os.path.abspath("")+"/corpus.conll"}")
    os.system(f"bash callMaltParser.sh {os.path.abspath("")+"/corpus.conll"}")
else:
    os.system(f"bash callMaltParser.sh {baselineFilePath}")
malt_parser = CoNLL.conll2doc(os.path.abspath("")+"/maltParserOut.conll")

/home/ivan/Desktop/Uni/TFG/RepositorioTFG/corpus.conll


-----------------------------------------------------------------------------
                          MaltParser 1.9.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Wed Mar 05 17:46:57 CET 2025
  Transition system    : Non-Projective
  Parser configuration : Covington with allow_root=true and allow_shift=false
  Data Format          : /espmalt-1.0/conllx.xml
.          	      1	      2s	    145MB
.          	     10	      2s	    145MB
.          	    100	      3s	    190MB
..........	   1000	     12s	    365MB
.......    	   1721	     15s	    304MB
Parsing time: 00:00:14 (14550 ms)
Finished: Wed Mar 05 17:47

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/maltParserOut.conll


In [6]:
print("Baseline: ", len(list(baseline.sentences)), "sentences")
#print("Malt Parser: ", len(list(malt_parser.sentences)), "sentences")
print(f"Spacy: {len(list(spacy_output.sents))} sentences. Baseline difference: ", (len(list(spacy_output.sents))/len(list(baseline.sentences))-1)*100, "%")
print(f"Stanza: {len(list(stanza_output.sentences))} sentences. Baseline difference: ", (len(list(stanza_output.sentences))/len(list(baseline.sentences))-1)*100, "%")
print(f"freeling: {len(freeling_parser)} sentences. Baseline difference: ", (len(freeling_parser)/len(list(baseline.sentences))-1)*100, "%")

Baseline:  1721 sentences
Spacy: 1714 sentences. Baseline difference:  -0.40674026728646506 %
Stanza: 1724 sentences. Baseline difference:  0.17431725740848503 %
freeling: 1495 sentences. Baseline difference:  -13.13190005810575 %


Sentence split is inconsistent between stanza, spacy and especially freeling when compared to the baseline, which is a real problem. This highlights the difficulty of comparing syntactic parsers, as there are many more systems that play a role such as PoS tagger and sentencizer. A different approach is required, maybe disabling the sentencizer and processing the corpus sentence by sentence?